# No Terminal

* pip install --upgrade pip
* pip install kafka-python
* pip install newsapi-python

# Bibliotecas

In [0]:
import pyspark.pandas as ps
import pandas as pd
import numpy as np
from pyspark.sql import functions as F
from kafka import KafkaConsumer
import datetime
import pickle

# Pipeline

In [0]:
class Data_Pipeline_API:
    def __init__(self) -> None:
        self.__df = None
        self.__df_parquet = None

    
    def load_api(self, data) -> None:
        self.__df = ps.DataFrame(data)
        sources = []
        for aux in self.__df['source'].to_numpy():
            if isinstance(aux, dict):
                sources.append(aux['name'])
            else:
                sources.append(None)
        self.__df['source'] = sources


    # Transformar os dados .parquet em DataFrame
    def load_parquet(self, name) -> None:
        file = f"{name}.parquet"
        self.__df_parquet = ps.read_parquet(f"/FileStore/Projeto/data/{file}")


    # Concatenando o DataFrame com o arquivo parquet
    # Mantém apenas colunas únicas
    def save_data(self, name) -> None:
        file = f"{name}.parquet"
        if file in [y.name.strip() for y in dbutils.fs.ls("/FileStore/Projeto/data/")] or f"{file}/" in [y.name.strip() for y in dbutils.fs.ls("/FileStore/Projeto/data/")]:
            self.load_parquet(name)
            self.__df = ps.concat([self.__df_parquet, self.__df]).drop_duplicates().reset_index().drop(columns=["index"])
        self.__df.to_parquet(f"/FileStore/Projeto/data/{file}", mode='overwrite')


    def data_cleaning(self):
        # Remove coluna de url de imagem de capa
        self.__df = self.__df_parquet.drop(columns=['urlToImage'])
        
        #Alterar o nome das colunas
        self.__df = self.__df.rename(columns={'author': 'Autor',
                                              'title': 'Título',
                                              'description': 'Descrição',
                                              'url' : 'URL',
                                              'publishedAt': 'Data Publicação',
                                              'content': 'Conteúdo',
                                              'source': 'Fonte'})

        # Organizar colunas
        df_order = ['Data Publicação', 'Título', 'Autor', 'Descrição', 'URL', 'Fonte', 'Conteúdo']
        self.__df = self.__df[df_order]

        # Ajuste do dia de publicação
        data = self.__df['Data Publicação'].to_numpy()
        date_df = pd.DataFrame(data = data)
        date_df.columns = ['Data Publicação']
        date_df['Data Publicação'] = pd.to_datetime(date_df['Data Publicação'])
        date_df['Data Publicação'] = date_df['Data Publicação'].dt.strftime('%Y/%m/%d')

        for i in range(self.__df.shape[0]):
            self.__df.loc[i, 'Data Publicação'] = date_df.loc[i, 'Data Publicação']

        # Ordenar Datas
        self.__df = self.__df.sort_values('Data Publicação').reset_index().drop(columns=['index'])
        

    @property
    def df(self):
        return self.__df


    @property
    def df_parquet(self):
        return self.__df_parquet

# Consumer

In [0]:
consumer = KafkaConsumer("genomics-news", bootstrap_servers=["localhost:9092"])

for message in consumer:
    value = message.value
    articles_list = pickle.loads(value)

    if len(articles_list) != 0:
        now = datetime.datetime.now()
        now_min = now.minute
        
        # Instanciando Pipeline
        pipeline_data = Data_Pipeline_API()

        # Carregando dados da API
        pipeline_data.load_api(articles_list)

        # Salvando em Parquet
        pipeline_data.save_data("raw_data")

        print(f"raw_data atualizado em {now.strftime('%Y-%m-%d %H:%M:%S')}")

        if now_min % 30 == 0:

            # Importando DataFrame
            pipeline_data.load_parquet("raw_data")

            # Limpeza de dados
            pipeline_data.data_cleaning()

            # Salvando Parquet
            pipeline_data.save_data("cleaned_data")

            print(f"cleaned_data atualizado em {now.strftime('%Y-%m-%d %H:%M:%S')}")